In [2]:
from genespectrakg import *

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from genespectrakg.adapters.process_eggnog_data_input import *

In [4]:
species_names_ids = {
    'hsapiens': '9606', 
    "mmulatta": '9544',
    'cjacchus': '9483', # marmoset
    'ptroglodytes': '9598', # chimpanzee
    'ggorilla': '9593', # western gorilla # seems to be 9595 in ncbi
}

In [52]:
all_species_mapped = pd.read_csv("data/MTG_eggnog_ensembl_mapped_mammalia.csv", dtype=str)

In [81]:
cell_ontology = pd.read_csv("data/MTG_cell_type_to_ontology_broad_with_taxo_cs.csv", dtype=str)

In [54]:
from genespectrakg.adapters.process_genespectra_input import *

In [55]:
cell_ontology.head()

,subclass,cell_type_name,cell_ontology_id,tissue_name,uberon_tissue_id,ncbi_txid,species_scientific_name,broad_type,broad_type_2,broad_type_3,broad_taxo_cs
0,Astro,astrocyte of the cerebral cortex,CL:0002605,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Astro
1,Oligo,oligodendrocyte,CL:0000128,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Oligo
2,VLMC,vascular leptomeningeal cell,CL:4023051,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,VLMC
3,Micro-PVM,microglial cell,CL:0000129,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Micro-PVM
4,OPC,oligodendrocyte precursor cell,CL:0002453,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,OPC


In [56]:
human_classes = pd.read_csv(f"data/1TPM_lim_taxo_species/human_classes_taxo_cs_1TPM_ensembl_name.csv")
human_processed = preprocess_genespectra_output(human_classes)

In [57]:
human_processed.head()
human_processed.shape

(8875, 11)

In [58]:
species_names_ids

{'hsapiens': '9606',
 'mmulatta': '9544',
 'cjacchus': '9483',
 'ptroglodytes': '9598',
 'ggorilla': '9593'}

In [59]:
import numpy as np

np.unique(human_processed['external_gene_name'].values)

array(['A1CF', 'A2M', 'A2ML1', ..., 'ZSCAN31', 'ZSWIM6', 'ZSWIM8-AS1'],
      dtype=object)

In [60]:
species_id_value = '9606'
## remember that after strsplit the species_id is a str
species_name = 'hsapiens'

print("Connecting to ensembl server")

server = biomart.BiomartServer("http://www.ensembl.org/biomart")
dataset = server.datasets[f'{species_name}_gene_ensembl']

attributes = ['ensembl_gene_id', 'external_gene_name']
filters = {'external_gene_name': np.unique(human_processed['external_gene_name'].values)}  

print("Downloading ensembl response")

response = dataset.search({'query': filters,
        'attributes': attributes,
        'mart_instance': 'ensembl'})

gene_to_id = pd.read_csv(response.url, sep='\t', header=None, names=attributes)
gene_to_id = gene_to_id.dropna()

Connecting to ensembl server


In [61]:
gene_to_id.head()
gene_to_id.shape

(48110, 2)

In [62]:
gene_to_id.loc[gene_to_id['external_gene_name'].isin(species_classes_new['external_gene_name'].values)]

,ensembl_gene_id,external_gene_name
141,ENSG00000286217,LINC00279
507,ENSG00000092377,TBL1Y
514,ENSG00000165246,NLGN4Y
520,ENSG00000231535,LINC00278
1059,ENSG00000240770,C21orf91-OT1
...,...,...
70660,ENSG00000162512,SDC3
70667,ENSG00000084628,NKAIN1
70683,ENSG00000078900,TP73
70689,ENSG00000160710,ADAR


In [63]:
species_classes_new = human_processed.merge(cell_ontology.loc[cell_ontology.ncbi_txid == '9606'], left_on='cell_type_name', right_on='broad_taxo_cs', how='left').reset_index(drop=True)

In [64]:
species_classes_new

,external_gene_name,specificity_category_type,specificity_category,specificity_score,distribution_catehory,n_expressed,mean_expression,max_expression,fraction_expressed,groups_expressed,...,cell_type_name_y,cell_ontology_id,tissue_name,uberon_tissue_id,ncbi_txid,species_scientific_name,broad_type,broad_type_2,broad_type_3,broad_taxo_cs
0,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,...,cerebral cortex endothelial cell,CL:1001602,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Endo
1,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,...,microglial cell,CL:0000129,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Micro-PVM
2,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,...,vascular leptomeningeal cell,CL:4023051,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,VLMC
3,ACER2,enriched,cell type enriched,10.189964,expressed in over 90%,10,26.560910,180.3695,100.0,Astro;CGE-derived;Endo;IT;MGE-derived;Micro-PV...,...,cerebral cortex endothelial cell,CL:1001602,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Endo
4,ACKR2,enriched,group enriched,5.114377,expressed in over 30%,6,4.361800,15.3874,60.0,Astro;Endo;Micro-PVM;OPC;Oligo;VLMC,...,astrocyte of the cerebral cortex,CL:0002605,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Astro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15882,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,astrocyte of the cerebral cortex,CL:0002605,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Astro
15883,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,oligodendrocyte precursor cell,CL:0002453,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,OPC
15884,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,oligodendrocyte,CL:0000128,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Oligo
15885,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,vascular leptomeningeal cell,CL:4023051,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,VLMC


In [65]:
species_classes_new.loc[species_classes_new['external_gene_name'].isin(gene_to_id['external_gene_name'].values)]

,external_gene_name,specificity_category_type,specificity_category,specificity_score,distribution_catehory,n_expressed,mean_expression,max_expression,fraction_expressed,groups_expressed,...,cell_type_name_y,cell_ontology_id,tissue_name,uberon_tissue_id,ncbi_txid,species_scientific_name,broad_type,broad_type_2,broad_type_3,broad_taxo_cs
0,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,...,cerebral cortex endothelial cell,CL:1001602,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Endo
1,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,...,microglial cell,CL:0000129,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Micro-PVM
2,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,...,vascular leptomeningeal cell,CL:4023051,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,VLMC
3,ACER2,enriched,cell type enriched,10.189964,expressed in over 90%,10,26.560910,180.3695,100.0,Astro;CGE-derived;Endo;IT;MGE-derived;Micro-PV...,...,cerebral cortex endothelial cell,CL:1001602,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Endo
4,ACKR2,enriched,group enriched,5.114377,expressed in over 30%,6,4.361800,15.3874,60.0,Astro;Endo;Micro-PVM;OPC;Oligo;VLMC,...,astrocyte of the cerebral cortex,CL:0002605,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Astro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15882,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,astrocyte of the cerebral cortex,CL:0002605,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Astro
15883,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,oligodendrocyte precursor cell,CL:0002453,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,OPC
15884,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,oligodendrocyte,CL:0000128,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Oligo
15885,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,...,vascular leptomeningeal cell,CL:4023051,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,VLMC


In [83]:
cell_ontology = cell_ontology.drop(['subclass', 'cell_type_name', 'cell_ontology_id', 'broad_type', 'broad_type_2', 'broad_type_3'], axis=1).drop_duplicates()

In [84]:
cell_ontology

,tissue_name,uberon_tissue_id,ncbi_txid,species_scientific_name,broad_taxo_cs
0,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Astro
1,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Oligo
2,middle temporal gyrus,UBERON:0002771,9606,hsapiens,VLMC
3,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Micro-PVM
4,middle temporal gyrus,UBERON:0002771,9606,hsapiens,OPC
5,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Endo
6,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-IT
10,middle temporal gyrus,UBERON:0002771,9606,hsapiens,CGE-derived
15,middle temporal gyrus,UBERON:0002771,9606,hsapiens,MGE-derived
19,middle temporal gyrus,UBERON:0002771,9606,hsapiens,IT


In [77]:
human_processed.shape

(8875, 11)

In [78]:
human_processed.merge(cell_ontology.loc[cell_ontology.ncbi_txid == '9606'], left_on='cell_type_name', right_on='broad_taxo_cs', how='left').reset_index(drop=True).drop_duplicates()

,external_gene_name,specificity_category_type,specificity_category,specificity_score,distribution_catehory,n_expressed,mean_expression,max_expression,fraction_expressed,groups_expressed,cell_type_name,tissue_name,uberon_tissue_id,ncbi_txid,species_scientific_name,broad_type,broad_type_2,broad_type_3,broad_taxo_cs
0,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,Endo,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Endo
1,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,Micro-PVM,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Micro-PVM
2,ABCB4,enriched,group enriched,7.955052,expressed in over 30%,4,3.577580,14.8364,40.0,CGE-derived;Endo;Micro-PVM;VLMC,VLMC,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,VLMC
3,ACER2,enriched,cell type enriched,10.189964,expressed in over 90%,10,26.560910,180.3695,100.0,Astro;CGE-derived;Endo;IT;MGE-derived;Micro-PV...,Endo,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Endo
4,ACKR2,enriched,group enriched,5.114377,expressed in over 30%,6,4.361800,15.3874,60.0,Astro;Endo;Micro-PVM;OPC;Oligo;VLMC,Astro,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Astro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9357,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,Astro,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Astro
9358,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,OPC,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,OPC
9359,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,Oligo,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,Oligo
9360,ZNF728,enriched,group enriched,10.416519,expressed in over 30%,4,1.253800,4.4360,40.0,Astro;OPC;Oligo;VLMC,VLMC,middle temporal gyrus,UBERON:0002771,9606,hsapiens,Non-neuronal,Non-neuronal,Non-neuronal,VLMC


In [85]:
def merge_genespectra_cl_ensembl(species_name_file:str, species_id:str, species_name:str):
    print(species_name_file)
    species_classes = pd.read_csv(f"data/1TPM_lim_taxo_species/{species_name_file}_classes_taxo_cs_1TPM_ensembl_name.csv")
    species_processed = preprocess_genespectra_output(species_classes)
    print(species_processed.shape)
    cell_ontology = pd.read_csv("data/MTG_cell_type_to_ontology_broad_with_taxo_cs.csv", dtype=str)
    cell_ontology = cell_ontology.drop(['subclass', 'cell_type_name', 'cell_ontology_id', 'broad_type', 'broad_type_2', 'broad_type_3'], axis=1).drop_duplicates()
    species_classes_new = species_processed.merge(cell_ontology.loc[cell_ontology.ncbi_txid == species_id], left_on='cell_type_name', right_on='broad_taxo_cs', how='left').reset_index(drop=True)
    print(species_classes_new.shape)
    print("Connecting to ensembl server")

    server = biomart.BiomartServer("http://www.ensembl.org/biomart")
    dataset = server.datasets[f'{species_name}_gene_ensembl']

    attributes = ['ensembl_gene_id', 'external_gene_name']
    filters = {'external_gene_name': np.unique(species_classes_new['external_gene_name'].values.astype(str))}   

    print("Downloading ensembl response")

    response = dataset.search({'query': filters,
            'attributes': attributes,
            'mart_instance': 'ensembl'})

    gene_to_id = pd.read_csv(response.url, sep='\t', header=None, names=attributes)
    gene_to_id = gene_to_id.dropna()
    print(gene_to_id.shape)
    species_classes_final = species_classes_new.merge(gene_to_id, left_on='external_gene_name', right_on='external_gene_name', how='left').dropna().reset_index(drop=True)
    print(species_classes_final.shape)
    species_classes_final.to_csv(f"data/1TPM_lim_taxo_species/{species_name_file}_classes_taxo_cs_1TPM_ensembl_name_processed_all.csv")
    

In [86]:
merge_genespectra_cl_ensembl(species_name_file='human', species_id='9606', species_name='hsapiens')

human
(8875, 11)
(8875, 16)
Connecting to ensembl server
(48110, 2)
(8872, 17)


In [87]:
merge_genespectra_cl_ensembl(species_name_file='gorilla', species_id='9593', species_name='ggorilla')
merge_genespectra_cl_ensembl(species_name_file='chimp', species_id='9598', species_name='ptroglodytes')
merge_genespectra_cl_ensembl(species_name_file='macaque', species_id='9544', species_name='mmulatta')
merge_genespectra_cl_ensembl(species_name_file='marmoset', species_id='9483', species_name='cjacchus')

gorilla
(11748, 11)
(11748, 16)
Connecting to ensembl server
(21227, 2)
(7466, 17)
chimp
(10867, 11)
(10867, 16)
Connecting to ensembl server
(23711, 2)
(6574, 17)
macaque
(11585, 11)
(11585, 16)
Connecting to ensembl server
(24114, 2)
(7267, 17)
marmoset
(8689, 11)
(8689, 16)
Connecting to ensembl server
(24804, 2)
(5452, 17)


In [88]:
species_name_file='gorilla'
species_id='9593'
species_name='ggorilla'

print(species_name_file)
species_classes = pd.read_csv(f"data/1TPM_lim_taxo_species/{species_name_file}_classes_taxo_cs_1TPM_ensembl_name.csv")
species_processed = preprocess_genespectra_output(species_classes)
print(species_processed.shape)
cell_ontology = pd.read_csv("data/MTG_cell_type_to_ontology_broad_with_taxo_cs.csv", dtype=str)
cell_ontology = cell_ontology.drop(['subclass', 'cell_type_name', 'cell_ontology_id', 'broad_type', 'broad_type_2', 'broad_type_3'], axis=1).drop_duplicates()
species_classes_new = species_processed.merge(cell_ontology.loc[cell_ontology.ncbi_txid == species_id], left_on='cell_type_name', right_on='broad_taxo_cs', how='left').reset_index(drop=True)
print(species_classes_new.shape)
print("Connecting to ensembl server")

server = biomart.BiomartServer("http://www.ensembl.org/biomart")
dataset = server.datasets[f'{species_name}_gene_ensembl']

attributes = ['ensembl_gene_id', 'external_gene_name']
filters = {'external_gene_name': np.unique(species_classes_new['external_gene_name'].values.astype(str))}   

print("Downloading ensembl response")

response = dataset.search({'query': filters,
        'attributes': attributes,
        'mart_instance': 'ensembl'})

gene_to_id = pd.read_csv(response.url, sep='\t', header=None, names=attributes)
gene_to_id = gene_to_id.dropna()
print(gene_to_id.shape)
species_classes_final = species_classes_new.merge(gene_to_id, left_on='external_gene_name', right_on='external_gene_name', how='left').dropna().reset_index(drop=True)
print(species_classes_final.shape)


gorilla
(11748, 11)
(11748, 16)
Connecting to ensembl server
(21227, 2)
(7466, 17)


In [89]:
species_classes_new.loc[~(species_classes_new['external_gene_name'].isin(gene_to_id['external_gene_name'].values)), :]

,external_gene_name,specificity_category_type,specificity_category,specificity_score,distribution_catehory,n_expressed,mean_expression,max_expression,fraction_expressed,groups_expressed,cell_type_name,tissue_name,uberon_tissue_id,ncbi_txid,species_scientific_name,broad_taxo_cs
0,AACSP1,enhanced,cell type enhanced,5.536236,expressed in over 30%,3,1.14278,6.3267,30.0,IT;Micro-PVM;Non-IT,Non-IT,middle temporal gyrus,UBERON:0002771,9593,ggorilla,Non-IT
4,ABCG2,enriched,cell type enriched,42.479515,expressed in over 90%,10,242.64389,2314.7344,100.0,Astro;CGE-derived;Endo;IT;MGE-derived;Micro-PV...,Endo,middle temporal gyrus,UBERON:0002771,9593,ggorilla,Endo
5,AC002091.1,enriched,cell type enriched,5.313555,expressed in single,1,0.27165,1.8895,10.0,Micro-PVM,Micro-PVM,middle temporal gyrus,UBERON:0002771,9593,ggorilla,Micro-PVM
6,AC002115.1,enriched,cell type enriched,7.835625,expressed in single,1,0.17435,1.1965,10.0,Endo,Endo,middle temporal gyrus,UBERON:0002771,9593,ggorilla,Endo
7,AC002398.2,enriched,cell type enriched,11.143310,expressed in single,1,0.12948,1.0575,10.0,Endo,Endo,middle temporal gyrus,UBERON:0002771,9593,ggorilla,Endo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11733,ZDHHC8P1,enriched,group enriched,4.417128,expressed in over 30%,7,5.94435,28.4512,70.0,Astro;CGE-derived;IT;MGE-derived;Micro-PVM;Non...,CGE-derived,middle temporal gyrus,UBERON:0002771,9593,ggorilla,CGE-derived
11738,ZFHX4-AS1,enhanced,cell type enhanced,5.612669,expressed in over 30%,5,4.24267,23.8127,50.0,Astro;CGE-derived;OPC;Oligo;VLMC,Astro,middle temporal gyrus,UBERON:0002771,9593,ggorilla,Astro
11744,ZNF385D-AS2,enriched,group enriched,18.885791,expressed in over 30%,3,2.48448,10.8997,30.0,CGE-derived;MGE-derived;Non-IT,CGE-derived,middle temporal gyrus,UBERON:0002771,9593,ggorilla,CGE-derived
11745,ZNF385D-AS2,enriched,group enriched,18.885791,expressed in over 30%,3,2.48448,10.8997,30.0,CGE-derived;MGE-derived;Non-IT,MGE-derived,middle temporal gyrus,UBERON:0002771,9593,ggorilla,MGE-derived
